In [ ]:
import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types
import spark.implicits._

In [ ]:
val df = spark.read
    .json("input/*.json")

In [ ]:
df.count()

In [ ]:
val schema = df.schema

In [ ]:
val df = spark.readStream
    .format("json")
    .schema(schema)
    .option("multiLine", "false")
    .option("cleanSource", "archive")
    .option("sourceArchiveDir", f"input_archive")
    .load("input/*.json")

In [ ]:
/*
df
    .writeStream
    .outputMode("append")
    .option("truncate", false)
    .format("console")
    .start()
    .awaitTermination()
    */

In [ ]:
val df2 = df.writeStream
    .option("path", "output")
    .option("checkpointLocation", "checkpoint")
    .trigger(org.apache.spark.sql.streaming.Trigger.ProcessingTime("5 seconds"))
    .toTable("houses")

In [ ]:
df2.processAllAvailable
//df2.stop
df2.awaitTermination

In [ ]:
val df3 = spark.read.table("houses")

In [ ]:
df3.count

In [ ]:
df2.stop